# SIF workflow for characterising synthetic proteins

# This is the development version 0.1

### First define the data sources - there should be a synthetic set of sequences as a single fasta file and single natural set of sequences as a fasta file. 

In [48]:
import os

#Scratch workspace
tmp=os.environ['TMP'] = '/home/neil/tmp/'

# Define the dataset names
synName = os.environ['SYNPROTS_NAME'] = 'ISP_1111_A1_short'
natName = os.environ['NATPROTS_NAME'] = 'sdr_pdg_filtered_short'

# Locations of the data folders

# Synthetic protein input data lives here
os.environ['SYNPROTS'] = '/home/neil/data/sif/synProts/ISP_1111_A1/'
# Natural protein input data lives here
os.environ['NATPROTS'] = '/home/neil/data/sif/natProts/zhen_sdrs/'

# Access the environment variable later in code
synProts = os.environ['SYNPROTS']
natProts = os.environ['NATPROTS']

# Synthetic protein structure output data lives here
synStrucs = os.environ['SYNSTRUCS'] = '/home/neil/data/sif/synStrucs/'

# Natural protein structure output data lives here 
natStrucs = os.environ['NATSTRUCS'] = '/home/neil/data/sif/natStrucs/'

# Synthetic protein comparison data lives here
os.environ['SYNPROTSCOMP'] = '/home/neil/data/sif/synProtComps/'
# Natural protein input data lives here
os.environ['NATPROTSCOMP'] = '/home/neil/data/sif/natProtComps/'
# Combined protein comparison data lives here
os.environ['COMPROTSCOMP'] = '/home/neil/data/sif/comProtComps/'
# Visualisation data lives here
natVisual=os.environ['NATVISUAL'] = '/home/neil/data/sif/visualisations/nat'
synVisual=os.environ['SYNVISUAL'] = '/home/neil/data/sif/visualisations/synthetic'
comVisual=os.environ['COMVISUAL'] = '/home/neil/data/sif/visualisations/natSyn'

# Access the environment variable later in code
synProtComps = os.environ['SYNPROTSCOMP']
natProtComps = os.environ['NATPROTSCOMP']
comProtComps = os.environ['COMPROTSCOMP']




In [ ]:
#Natural language library
!pip install nltk

In [ ]:
# Import necessary libraries
import nltk
import random
import uuid
from collections import Counter

# Download the brown corpus
nltk.download('brown')

# Get a list of English words
word_list = nltk.corpus.brown.words()

# Count the occurrences of each word
word_counter = Counter(word_list)

# Get the 5000 most common words
common_words = [word for word, _ in word_counter.most_common(5000)]

def generate_human_memorable_id():
    # Generate a unique ID using uuid
    unique_id = uuid.uuid4().int

    # Select two random words
    word1 = random.choice(common_words)
    word2 = random.choice(common_words)

    # Truncate the UUID to the last 6 digits for brevity and append it to the words
    human_memorable_id = f'{word1}-{word2}-{str(unique_id)[-6:]}'

    return human_memorable_id

# Print a unique, human-memorable ID
wf_id = generate_human_memorable_id()
print("Workflow ID is:", wf_id)


### Structural Predictions using esmFold

**Fold the synthetic proteins**

In [ ]:
!pip install py3Dmol
!pip install biopython
!pip install transformers
!pip install torch
!pip install pandas
!pip install accelerate

# Build the complete paths first
fasta_path = f"{synProts}{synName}.fasta"
output_path = f"{synStrucs}{wf_id}/{synName}"

# Use the paths in the command
!python /home/neil/projects-dep/esmFold/esmFold.py {fasta_path} {output_path}
print("/home/neil/projects-dep/esmFold/esmFold.py", fasta_path, output_path)

**Fold the natural proteins**

In [ ]:
# Build the complete paths first
fasta_path = f"{natProts}{natName}.fasta"
output_path = f"{natStrucs}{wf_id}/{natName}"

# Use the paths in the command
!python /home/neil/projects-dep/esmFold/esmFold.py {fasta_path} {output_path}
print("/home/neil/projects-dep/esmFold/esmFold.py", fasta_path, output_path)

# Building the SSNs



We are going to do this manually over a terminal to TBone as it can take a while.

1) SSH into TBone e.g. ssh neil@92.40.34.250
2) conda activate nf-needleall-ava
3) export NXF_VER=22.10.0
4) run the command generated below so that things end up in the right directory

## Synthetic Proteins

In [ ]:
fasta_path = f"{synProts}{synName}.fasta"
comp_output_path_ssn = f"{synProtsComp}ssn/{synName}/{wf_id}"
print ("nextflow run ravenlocke/nf-needleall-ava --infile", fasta_path, "--outdir", comp_output_path_ssn, "--threshold 0.4 &")

## Natural Proteins

In [ ]:
fasta_path = f"{natProts}{natName}.fasta"
comp_output_path_ssn = f"{natProtsComp}ssn/{natName}/{wf_id}"
print ("nextflow run ravenlocke/nf-needleall-ava --infile", fasta_path, "--outdir", comp_output_path_ssn, "--threshold  0.4 &")

# Building the Structural SNs (STSNs)

## Synthetic Proteins against themselves

Using structComp: Uses foldseek to compare one set of PDB files to another 
Usage: python3 structComp.py <query_path> <target_path> <results_file>



In [29]:

# Build the complete paths first
query_path = f"{synStrucs}{wf_id}/{synName}"
target_path = f"{synStrucs}{wf_id}/{synName}"
results_file = f"{synProtComps}stsn/{synName}/{wf_id}/{synName}-all.tsv"
!mkdir {synProtComps}stsn/{synName}
!mkdir {synProtComps}stsn/{synName}/{wf_id}
syn_comp_results = results_file

# Use the paths in the command
!python /home/neil/projects-dev/structComp/structComp.py {query_path} {target_path} {results_file}
print("/home/neil/projects-dev/structComp/structComp.py", query_path, target_path, results_file)


mkdir: cannot create directory ‘/home/neil/data/sif/synProtComps/stsn/ISP_1111_A1_short’: File exists
mkdir: cannot create directory ‘/home/neil/data/sif/synProtComps/stsn/ISP_1111_A1_short/actions-done-253755’: File exists
/home/neil/data/sif/synProtComps/stsn/ISP_1111_A1_short/actions-done-253755/ISP_1111_A1_short-all.tsv exists and will be overwritten
easy-search /home/neil/data/sif/synStrucs/actions-done-253755/ISP_1111_A1_short /home/neil/data/sif/synStrucs/actions-done-253755/ISP_1111_A1_short /home/neil/data/sif/synProtComps/stsn/ISP_1111_A1_short/actions-done-253755/ISP_1111_A1_short-all.tsv ./tmp --format-mode 4 

MMseqs Version:              	7.04e0ec8
Seq. id. threshold           	0
Coverage threshold           	0
Coverage mode                	0
Max reject                   	2147483647
Max accept                   	2147483647
Add backtrace                	false
TMscore threshold            	0
TMalign hit order            	0
TMalign fast                 	1
Preload mode       

/home/neil/projects-dev/structComp/structComp.py /home/neil/data/sif/synStrucs/actions-done-253755/ISP_1111_A1_short /home/neil/data/sif/synStrucs/actions-done-253755/ISP_1111_A1_short /home/neil/data/sif/synProtComps/stsn/ISP_1111_A1_short/actions-done-253755/ISP_1111_A1_short-all.tsv


## Natural Proteins against themselves

In [ ]:
# Build the complete paths first
query_path = f"{natStrucs}{wf_id}/{natName}"
target_path = f"{natStrucs}{wf_id}/{natName}"
results_file = f"{natProtComps}stsn/{natName}/{wf_id}/{natName}-all.tsv"
!mkdir {natProtComps}stsn/{natName}
!mkdir {natProtComps}stsn/{natName}/{wf_id}

nat_comp_results = results_file
# Use the paths in the command
!python /home/neil/projects-dev/structComp/structComp.py {query_path} {target_path} {results_file}
print("/home/neil/projects-dev/structComp/structComp.py", query_path, target_path, results_file)

## Natural Proteins and Synth proteins all against all

In [ ]:
#Copy the natural proteins to the tmp directory
import shutil
source_directory = f"{natStrucs}{wf_id}/{natName}"
destination_directory = f"{tmp}{wf_id}"
shutil.copytree(source_directory, destination_directory,dirs_exist_ok=True)

#Copy the synthetic proteins to the tmp directory
source_directory = f"{synStrucs}{wf_id}/{synName}"
shutil.copytree(source_directory, destination_directory,dirs_exist_ok=True)
#Set the query and target directory to this combined directory
query_path=source_directory
target_path=source_directory
!mkdir {comProtComps}stsn/{wf_id}
#Set the results file to the right directory
results_file = f"{comProtComps}stsn/{wf_id}/{natName}-vs-{synName}-all.tsv"
comb_comp_results = results_file

!python /home/neil/projects-dev/structComp/structComp.py {query_path} {target_path} {results_file}
print("/home/neil/projects-dev/structComp/structComp.py", query_path, target_path, results_file)



# Visualising the Structural SNs (STSNs)


### usage: structCompVis.py [-h] input_file output_shortened_file output_cytoscape_file output_gml_file cutoff

In [49]:
results_file=syn_comp_results 
results_file_short=f"{synVisual}/{wf_id}/{synName}_short.tsv"
cyto_file_sif=f"{synVisual}/{wf_id}/{synName}.sif"
cyto_file_gml=f"{synVisual}/{wf_id}/{synName}.gml"
cutoff=0.3

In [50]:
!mkdir {synVisual}/{wf_id}

In [52]:
print("/home/neil/projects-dev/structComp/structCompVis.py",results_file, results_file_short, cyto_file_sif, cyto_file_gml, cutoff)
!python3 /home/neil/projects-dev/structComp/structCompVis.py {results_file} {results_file_short} {cyto_file_sif} {cyto_file_gml} {cutoff}


/home/neil/projects-dev/structComp/structCompVis.py /home/neil/data/sif/synProtComps/stsn/ISP_1111_A1_short/actions-done-253755/ISP_1111_A1_short-all.tsv /home/neil/data/sif/visualisations/synthetic/actions-done-253755/ISP_1111_A1_short_short.tsv /home/neil/data/sif/visualisations/synthetic/actions-done-253755/ISP_1111_A1_short.sif /home/neil/data/sif/visualisations/synthetic/actions-done-253755/ISP_1111_A1_short.gml 0.3
Received arguments: Namespace(input_file='/home/neil/data/sif/synProtComps/stsn/ISP_1111_A1_short/actions-done-253755/ISP_1111_A1_short-all.tsv', output_shortened_file='/home/neil/data/sif/visualisations/synthetic/actions-done-253755/ISP_1111_A1_short_short.tsv', output_cytoscape_file='/home/neil/data/sif/visualisations/synthetic/actions-done-253755/ISP_1111_A1_short.sif', output_gml_file='/home/neil/data/sif/visualisations/synthetic/actions-done-253755/ISP_1111_A1_short.gml', cutoff=0.3)
Successfully written to /home/neil/data/sif/visualisations/synthetic/actions-done